<a href="https://colab.research.google.com/github/BalaSree2005/idp_3.2/blob/main/idp_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to the datasets
train_path = '/content/drive/MyDrive/parkinson/Dataset/train'
valid_path = '/content/drive/MyDrive/parkinson/Dataset/valid'
test_path = '/content/drive/MyDrive/parkinson/Dataset/test'

# Custom Dataset Class
class ParkinsonDataset(Dataset):
    def __init__(self, directory):
        self.images = []
        self.labels = []
        for label, subdir in enumerate(os.listdir(directory)):
            subdir_path = os.path.join(directory, subdir)
            if os.path.isdir(subdir_path):
                for img_name in os.listdir(subdir_path):
                    if img_name.lower().endswith(('png', 'jpg', 'jpeg')):
                        img_path = os.path.join(subdir_path, img_name)
                        img = Image.open(img_path).resize((150, 150))  # Resize to 150x150
                        self.images.append(np.array(img) / 255.0)  # Normalize
                        self.labels.append(label)

        self.images = np.array(self.images).astype(np.float32)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx].flatten(), dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

# Load data using DataLoader with larger batch size
batch_size = 128  # Increased batch size for better GPU utilization
train_dataset = ParkinsonDataset(train_path)
valid_dataset = ParkinsonDataset(valid_path)
test_dataset = ParkinsonDataset(test_path)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the neural network model
class ParkinsonNet(nn.Module):
    def __init__(self):
        super(ParkinsonNet, self).__init__()
        self.fc1 = nn.Linear(150 * 150 * 3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # Assuming 2 classes (0 and 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = ParkinsonNet().to(device)

# Enable DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scaler = torch.cuda.amp.GradScaler()  # Enable Mixed Precision Training

# Training loop
def train_model(model, criterion, optimizer, train_loader, valid_loader, epochs=10):
    train_losses = []
    val_losses = []
    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # Mixed precision training
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        train_losses.append(running_loss / len(train_loader))

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)

                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                val_loss += loss.item()

        val_losses.append(val_loss / len(valid_loader))

        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

    total_time = time.time() - start_time
    print(f"Training completed in {total_time:.2f} seconds")
    return train_losses, val_losses

# Train the model
print("Training Neural Network...")
train_losses, val_losses = train_model(model, criterion, optimizer, train_loader, valid_loader, epochs=10)

# Evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, axis=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy, all_preds, all_labels

test_accuracy, test_preds, test_labels = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Confusion Matrix and Classification Report
test_cm = confusion_matrix(test_labels, test_preds)
print("Test Confusion Matrix:")
print(test_cm)
print("Classification Report:")
print(classification_report(test_labels, test_preds))

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(test_labels), yticklabels=np.unique(test_labels))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Test Confusion Matrix')
plt.show()

# Plot training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()


In [ ]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

# Set device (CPU only)
device = torch.device("cpu")
print(f"Using device: {device}")

# Paths to the datasets
train_path = '/content/drive/MyDrive/parkinson/Dataset/train'
valid_path = '/content/drive/MyDrive/parkinson/Dataset/valid'
test_path = '/content/drive/MyDrive/parkinson/Dataset/test'

# Custom Dataset Class
class ParkinsonDataset(Dataset):
    def __init__(self, directory):
        self.images = []
        self.labels = []
        for label, subdir in enumerate(os.listdir(directory)):
            subdir_path = os.path.join(directory, subdir)
            if os.path.isdir(subdir_path):
                for img_name in os.listdir(subdir_path):
                    if img_name.lower().endswith(('png', 'jpg', 'jpeg')):
                        img_path = os.path.join(subdir_path, img_name)
                        img = Image.open(img_path).resize((150, 150))  # Resize to 150x150
                        self.images.append(np.array(img) / 255.0)  # Normalize
                        self.labels.append(label)

        self.images = np.array(self.images).astype(np.float32)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx].flatten(), dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

# Load data using DataLoader
batch_size = 128
train_dataset = ParkinsonDataset(train_path)
valid_dataset = ParkinsonDataset(valid_path)
test_dataset = ParkinsonDataset(test_path)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Define the neural network model
class ParkinsonNet(nn.Module):
    def __init__(self):
        super(ParkinsonNet, self).__init__()
        self.fc1 = nn.Linear(150 * 150 * 3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # Assuming 2 classes (0 and 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = ParkinsonNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, criterion, optimizer, train_loader, valid_loader, epochs=10):
    train_losses = []
    val_losses = []
    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_losses.append(running_loss / len(train_loader))

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_losses.append(val_loss / len(valid_loader))

        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

    total_time = time.time() - start_time
    print(f"Training completed in {total_time:.2f} seconds")
    return train_losses, val_losses

# Train the model
print("Training Neural Network...")
train_losses, val_losses = train_model(model, criterion, optimizer, train_loader, valid_loader, epochs=10)

# Evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, axis=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy, all_preds, all_labels

test_accuracy, test_preds, test_labels = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Confusion Matrix and Classification Report
test_cm = confusion_matrix(test_labels, test_preds)
print("Test Confusion Matrix:")
print(test_cm)
print("Classification Report:")
print(classification_report(test_labels, test_preds))

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(test_labels), yticklabels=np.unique(test_labels))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Test Confusion Matrix')
plt.show()

# Plot training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()


In [ ]:
import torch
import time
import os
import multiprocessing

# Check GPU Availability
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    print(f"Number of GPUs Available: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPUs available, running on CPU.")

# Check Number of CPU Cores
num_cpu_cores = multiprocessing.cpu_count()
print(f"Number of CPU Cores Available: {num_cpu_cores}")

# Check Parallelism using Batch Size Performance
def measure_time(batch_size):
    start_time = time.time()

    # Dummy Tensor Computation to Simulate Training
    x = torch.randn(batch_size, 150 * 150 * 3).to("cuda" if num_gpus > 0 else "cpu")
    model = torch.nn.Linear(150 * 150 * 3, 128).to("cuda" if num_gpus > 0 else "cpu")

    for _ in range(10):  # Simulate 10 forward passes
        y = model(x)

    return time.time() - start_time

# Measure time for different batch sizes
batch_sizes = [32, 64, 128, 256]
for batch in batch_sizes:
    elapsed_time = measure_time(batch)
    print(f"Batch Size: {batch}, Time Taken: {elapsed_time:.4f} sec")



In [ ]:
import torch
import time
from torchinfo import summary

# Define the ParkinsonNet Model
class ParkinsonNet(torch.nn.Module):
    def __init__(self):
        super(ParkinsonNet, self).__init__()
        self.fc1 = torch.nn.Linear(150 * 150 * 3, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 2)  # Assuming 2 classes (0 and 1)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Initialize Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ParkinsonNet().to(device)

# 1. Display Model Summary (FLOPs, Parameters)
print("\n==== Model Summary ====")
summary(model, input_size=(1, 150 * 150 * 3))

# 2. Measure Execution Time and Parallel Instructions
def measure_parallel_execution(model, batch_size, input_size):
    model.to(device)
    model.eval()

    dummy_input = torch.randn(batch_size, input_size).to(device)

    start_time = time.time()
    with torch.no_grad():
        _ = model(dummy_input)
    end_time = time.time()

    elapsed_time = end_time - start_time
    num_operations = 150 * 150 * 3 * batch_size  # Estimate based on input size

    print(f"Batch Size: {batch_size}, Time Taken: {elapsed_time:.6f} sec, Estimated Instructions: {num_operations / elapsed_time:.2e} instr/sec")
    return elapsed_time, num_operations

# Test with Different Batch Sizes
print("\n==== Parallel Execution Analysis ====")
batch_sizes = [1, 32, 64, 128, 256]
for batch_size in batch_sizes:
    measure_parallel_execution(model, batch_size, 150 * 150 * 3)


In [ ]:
import torch
import time
from torchinfo import summary

# Force CPU Execution
device = "cpu"

# Define the ParkinsonNet Model
class ParkinsonNet(torch.nn.Module):
    def __init__(self):
        super(ParkinsonNet, self).__init__()
        self.fc1 = torch.nn.Linear(150 * 150 * 3, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 2)  # Assuming 2 classes (0 and 1)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Initialize Model on CPU
model = ParkinsonNet().to(device)

# 1. Display Model Summary
print("\n==== Model Summary ====")
summary(model, input_size=(1, 150 * 150 * 3), device="cpu")

# 2. Measure Execution Time and Instructions Per Second on CPU
def measure_cpu_execution(model, batch_size, input_size):
    model.to(device)
    model.eval()

    dummy_input = torch.randn(batch_size, input_size).to(device)

    start_time = time.time()
    with torch.no_grad():
        _ = model(dummy_input)
    end_time = time.time()

    elapsed_time = end_time - start_time
    num_operations = 150 * 150 * 3 * batch_size  # Estimate based on input size

    ips = num_operations / elapsed_time if elapsed_time > 0 else 0  # Instructions per second
    print(f"Batch Size: {batch_size}, Time Taken: {elapsed_time:.6f} sec, Estimated IPS: {ips:.2e} instr/sec")
    return elapsed_time, ips

# Test with Different Batch Sizes on CPU
print("\n==== CPU Execution Analysis ====")
batch_sizes = [1, 32, 64, 128, 256]
for batch_size in batch_sizes:
    measure_cpu_execution(model, batch_size, 150 * 150 * 3)


In [ ]:
import torch
from torchviz import make_dot

# Sample input tensor
sample_input = torch.randn(1, 150 * 150 * 3).to(device)

# Forward pass to get model output
output = model(sample_input)

# Generate task graph
task_graph = make_dot(output, params=dict(model.named_parameters()))

# Render and save graph
task_graph.format = "png"
task_graph.render("task_graph")

# Display graph
from IPython.display import Image
Image("task_graph.png")
